In [12]:
import pdfplumber
from pathlib import Path
import os
import re
import shutil
import pandas as pd

CAMINHO_RAIZ = r"C:\Users\julianobs\OneDrive - Suzano S A\Area Dev\GitHub\Leitura-faturas-energisa"

class RegexExtractor:
    regex = {
        "uc_simples": [r"CÓDIGO:\s*(\d+)\d", r"UC:\s*(\d+)"],
        "uc": [r"\d{2}/\d{7}-\d",r"\d{7}-\d"],
        # "venc": [r'Protocolo de Autorização:.*?(\d{2}/\d{2}/\d{4})\s*R\$', r"\d{4}-\d{2}-\d\s+(\d{2}/\d{2}/\d{4})"],
        # "emis": [r"EMISS[ÃA]O\s*[:\-]?\s*(\d{2}/\d{2}/\d{4})", r"DATA EMISSÂO/APRESENTAÇÂO:(\d{2}/\d{2}/\d{4})"],
        # "nf": [r"NOTA FISCAL Nº[:\-]?\s*(\d{3}\.\d{3}\.\d{3})", r"NOTA FISCAL N°[:\-]?\s*(\d{3}\.\d{3}\.\d{3})"],
        # "cb": [r"(\d{11}-\d\s\d{11}-\d\s\d{11,13}-\d\s\d{11}-\d)", r"(\d{5}\.\d{5}\s\d{5}\.\d{6}\s\d{5}\.\d{6}\s\d\s\d{14})"],
        # "val": [r"R\$ (\d{1,3},\d{2})"]
    }

    @classmethod
    def extrair(cls, chave, texto):
        for rgx in cls.regex.get(chave, []):
            match = re.search(rgx, texto)
            if match:
                try:
                    return match.group(1)
                except:
                    return match.group()
        return "Não encontrado"

class PDFProcessor:
    def __init__(self, caminho_pdf):
        self.caminho_pdf = caminho_pdf
        self.texto = self._extrair_texto()

    def _extrair_texto(self):
        texto = ''
        with pdfplumber.open(self.caminho_pdf) as pdf:
            for page in pdf.pages:
                texto += page.extract_text() + '\n'
        print(texto)
        return texto

    def extrair_dados(self):
        texto_unico = ' '.join(self.texto.split('\n'))
        return {
            "Item": self.caminho_pdf.name,
            "uc": RegexExtractor.extrair("uc", texto_unico),
            "uc_simples": RegexExtractor.extrair("uc_simples", texto_unico),
            # "venc": RegexExtractor.extrair("venc", texto_unico),
            # "emissao": RegexExtractor.extrair("emis", texto_unico),
            # "valor_total": RegexExtractor.extrair("val", texto_unico),
            # "nota_fiscal": RegexExtractor.extrair("nf", texto_unico),
            # "cod_barras": RegexExtractor.extrair("cb", texto_unico)
        }

class GerenciadorFaturas:
    def __init__(self, raiz):
        self.pasta_pendentes = Path(raiz) / "faturas_pendentes"
        self.pasta_sucesso = Path(raiz) / "faturas_sucesso"
        self.pasta_erro = Path(raiz) / "faturas_erro"
        self.lista_resultados = []

    def mover_arquivo(self, arquivo, sucesso):
        origem = self.pasta_pendentes / arquivo
        destino = self.pasta_sucesso if sucesso else self.pasta_erro
        shutil.move(origem, destino)

    def processar_faturas(self):
        for item in os.listdir(self.pasta_pendentes):
            caminho_pdf = self.pasta_pendentes / item
            processor = PDFProcessor(caminho_pdf)
            dados = processor.extrair_dados()
            self.lista_resultados.append(dados)

            if "Não encontrado" in dados.values():
                print(f"[ERRO] {item} - Campo não encontrado")
                self.mover_arquivo(item, False)
            else:
                print(f"[OK] {item} - Todos os campos extraídos")
                self.mover_arquivo(item, True)
        
        df = pd.DataFrame(self.lista_resultados)
        df.to_excel(Path(CAMINHO_RAIZ) / "faturas.xlsx", index=False)
        return df



In [13]:
gerenciador = GerenciadorFaturas(CAMINHO_RAIZ)
df = gerenciador.processar_faturas()

DANF3E - DOCUMENTO AUXILIAR DA NOTA FISCAL DE ENERGIA ELÉTRICA ELETRÔNICA
ENERGISA MATO GROSSO DO SUL - DISTR. DE ENERGIA S.A.
Av. Gury Marques, 8000
Campo Grande/MS - CEP 79072-900
CNPJ 15.413.826/0001-50 Insc. Est. 28.105.553-0
DOMICÍLIO DE ENTREGA
CONTA FACIL RIBAS DO RIO PARDO
RIBAS DO RIO PARDO MS (AG: 23)
ROTEIRO:055 - 0023 - 550 - 0620
MATRÍCULA: 2000537-2025-8-5
DOM. BANC.: DOM. ENT.:9999061001
Classificação: MTC-CONVENCIONAL BAIXA TENSÃO / B1
RESIDENCIAL / RESIDENCIAL LIGAÇÃO:TRIFASICO
TENSÃO NOMINAL EM VOLTS DISP: Lim. Min.:117 Lim. Max.:7 a
SUZANO S A SOCIEDADE ANONIMA ABERTA
08/07/2025 07/08/2025 30 09/09/2025
FAZENDA CASSUNUNGA, S/N - 24.423.03.456506 - 79180000
RURAL
RIBAS DO RIO PARDO (AG: 23)
CNPJ/CPF/RANI: 1X.XXX.XXX/XXX1-55
Insc. Est.:
Preço unit (R$) PIS/ Base Calc.% Alíq. ICMS Tarifa Base de Alíquota Valor
Itens da Fatura Unid. Quant. com tributos Valor (R$)COFINS (R$) ICMS (R$) ICMS (R$) Unit (R$) Tributo Cálc.(R$) (%) (R$)
Consumo em kWh 1,099590 3.407,67 107,13 3

In [15]:
df

,Item,uc,uc_simples
0,08.2025 - Energisa - Inst 102000537-7.pdf,10/2000537-7,Não encontrado
1,08.2025 - Energisa - Inst 102041018-9.pdf,10/2041018-9,Não encontrado
2,08.2025 - Energisa - Inst 102041025-4.pdf,10/2041025-4,Não encontrado
3,08.2025 - Energisa - Inst 102393895-4.pdf,10/2393895-4,Não encontrado
4,08.2025 - Energisa - Inst 102616081-2.pdf,10/2616081-2,0002616081
...,...,...,...
121,11.2025 - Energisa - Inst 102587682-2_Hospital...,10/2587682-2,Não encontrado
122,11.2025 - Energisa - Inst 102616081-2.pdf,10/2616081-2,0002616081
123,11.2025 - Energisa - Inst 103510949-5.pdf,10/3510949-5,Não encontrado
124,11.2025 - ENERGISA FZD TTG - Inst 103559273-2.pdf,10/3559273-2,Não encontrado
